In [1]:
import pandas as pd  
import numpy as np 

In [2]:
df=pd.read_csv('Churn_Modelling (1).csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
df.drop(columns=['RowNumber','CustomerId','Surname','Geography'],inplace=True)
df.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Gender']=le.fit_transform(df['Gender'])
df.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,42,2,0.00,1,1,1,101348.88,1
1,608,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,39,1,0.00,2,0,0,93826.63,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
df[['CreditScore', 'EstimatedSalary', 'Balance', 'Age','Tenure','NumOfProducts']] = sc.fit_transform(df[['CreditScore', 'EstimatedSalary', 'Balance', 'Age','Tenure','NumOfProducts']])
df.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,-0.326221,0,0.293517,-1.041760,-1.225848,-0.911583,1,1,0.021886,1
1,-0.440036,0,0.198164,-1.387538,0.117350,-0.911583,0,1,0.216534,0
2,-1.536794,0,0.293517,1.032908,1.333053,2.527057,1,0,0.240687,1
3,0.501521,0,0.007457,-1.387538,-1.225848,0.807737,0,0,-0.108918,0
4,2.063884,0,0.388871,-1.041760,0.785728,-0.911583,1,1,-0.365276,0


In [7]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [8]:
x.shape

(10000, 9)

In [9]:
y.shape

(10000,)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [11]:
from sklearn.ensemble import StackingClassifier 
from sklearn.linear_model import LogisticRegression,RidgeClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report 
import joblib 
bc=[
    ('ridge',RidgeClassifier()),
    ('lr',LogisticRegression(max_iter=1000)),
    ('dt',DecisionTreeClassifier(max_depth=5,random_state=42)),
    ('rf', RandomForestClassifier(max_depth=5, random_state=42)),
    ('svc',SVC(probability=True)),
    ('gbc',GradientBoostingClassifier(max_depth=5,random_state=42)),
    ('xgb',XGBClassifier(max_depth=5,random_state=42,use_label_encoder=False,eval_metric='logloss'))
]
mc=LogisticRegression()
stacking_classifier=StackingClassifier(
    estimators=bc,
    final_estimator=mc
)
pipe=Pipeline([
    ('stacking',stacking_classifier)
])
pipe.fit(x_train, y_train)
y_pred=pipe.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred))
print("Precision:",precision_score(y_test, y_pred))
print("Recall:",recall_score(y_test,y_pred))
print("F1 Score:",f1_score(y_test, y_pred))
print("\nClassification Report:\n",classification_report(y_test, y_pred))


C:\Users\Hi\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [21:59:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Hi\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:00:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Hi\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:00:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarn

Accuracy: 0.862
Precision: 0.7510729613733905
Recall: 0.44529262086513993
F1 Score: 0.5591054313099042

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.75      0.45      0.56       393

    accuracy                           0.86      2000
   macro avg       0.81      0.70      0.74      2000
weighted avg       0.85      0.86      0.85      2000



In [12]:
from sklearn.ensemble import BaggingClassifier 
be=[
    ('ridge',RidgeClassifier()),
    ('lr',LogisticRegression(max_iter=1000)),
    ('dt',DecisionTreeClassifier(max_depth=5,random_state=42)),
    ('rf', RandomForestClassifier(max_depth=5, random_state=42)),
    ('svc',SVC(probability=True)),
    ('gbc',GradientBoostingClassifier(max_depth=5,random_state=42)),
    ('xgb',XGBClassifier(max_depth=5,random_state=42,use_label_encoder=False,eval_metric='logloss'))
    
]

for name,estimator in be:
    bm=BaggingClassifier(
        estimator=estimator, 
        n_estimators=10,
        random_state=42
    )
    bm.fit(x_train,y_train)
    y_pred1=bm.predict(x_test)
    print(f"\n{name.upper()} as base estimator")
    print("Accuracy:",accuracy_score(y_test, y_pred1))
    print("Precision:",precision_score(y_test, y_pred1,average='binary'))
    print("Recall:",recall_score(y_test,y_pred1,average='binary'))
    print("F1 Score:",f1_score(y_test, y_pred1,average='binary'))
    print("\nClassification Report:\n",classification_report(y_test, y_pred1))
    


RIDGE as base estimator
Accuracy: 0.8045
Precision: 0.5172413793103449
Recall: 0.07633587786259542
F1 Score: 0.13303769401330376

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.98      0.89      1607
           1       0.52      0.08      0.13       393

    accuracy                           0.80      2000
   macro avg       0.67      0.53      0.51      2000
weighted avg       0.75      0.80      0.74      2000


LR as base estimator
Accuracy: 0.812
Precision: 0.5726495726495726
Recall: 0.17048346055979643
F1 Score: 0.2627450980392157

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.97      0.89      1607
           1       0.57      0.17      0.26       393

    accuracy                           0.81      2000
   macro avg       0.70      0.57      0.58      2000
weighted avg       0.78      0.81      0.77      2000


DT as base estimator
Accuracy: 0.861
Pre

C:\Users\Hi\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:01:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Hi\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:01:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



XGB as base estimator
Accuracy: 0.859
Precision: 0.7283950617283951
Recall: 0.45038167938931295
F1 Score: 0.5566037735849056

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.73      0.45      0.56       393

    accuracy                           0.86      2000
   macro avg       0.80      0.70      0.74      2000
weighted avg       0.85      0.86      0.85      2000



In [13]:
joblib.dump(pipe, 'stacking_churn_classifier.pkl')

['stacking_churn_classifier.pkl']

In [14]:
import joblib
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
ada_boost = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=5, random_state=42), 
    n_estimators=50, 
    random_state=42
)

ada_boost.fit(x_train, y_train)
y_pred = ada_boost.predict(x_test)
print("\nAdaBoost with Decision Tree as base estimator")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='binary'))
print("Recall:", recall_score(y_test, y_pred, average='binary'))
print("F1 Score:", f1_score(y_test, y_pred, average='binary'))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
model_filename = 'adaboost_decision_tree_model.pkl'
joblib.dump(ada_boost, model_filename)
print(f"Model saved as {model_filename}")


C:\Users\Hi\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



AdaBoost with Decision Tree as base estimator
Accuracy: 0.8025
Precision: 0.4969512195121951
Recall: 0.41475826972010177
F1 Score: 0.4521497919556172

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.90      0.88      1607
           1       0.50      0.41      0.45       393

    accuracy                           0.80      2000
   macro avg       0.68      0.66      0.67      2000
weighted avg       0.79      0.80      0.80      2000

Model saved as adaboost_decision_tree_model.pkl


In [15]:
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

model = joblib.load('adaboost_decision_tree_model.pkl')
scaler = StandardScaler()

label_encoder = LabelEncoder()

new_user_data = {
    'CreditScore': 1456,
    'Gender': 'Male',
    'Age': 20,
    'Tenure': 3,
    'Balance': 10000,
    'NumOfProducts': 3,
    'HasCrCard': 'Yes',
    'IsActiveMember': 'Yes',
    'EstimatedSalary': 50000
}
gender_encoded = label_encoder.fit_transform(['Male', 'Female']) 
gender_value = label_encoder.transform([new_user_data['Gender']])[0]

has_cr_card = 1 if new_user_data['HasCrCard'] == 'Yes' else 0
is_active_member = 1 if new_user_data['IsActiveMember'] == 'Yes' else 0

user_input = pd.DataFrame({
    'CreditScore': [new_user_data['CreditScore']],
    'Gender': [gender_value],
    'Age': [new_user_data['Age']],
    'Tenure': [new_user_data['Tenure']],
    'Balance': [new_user_data['Balance']],
    'NumOfProducts': [new_user_data['NumOfProducts']],
    'HasCrCard': [has_cr_card],
    'IsActiveMember': [is_active_member],
    'EstimatedSalary': [new_user_data['EstimatedSalary']]
})

numerical_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
user_input[numerical_columns] = scaler.fit_transform(user_input[numerical_columns])

prediction = model.predict(user_input)

if prediction[0] == 1:
    print("Customer will exit")
else:
    print("Customer will not exit")


Customer will not exit


C:\Users\Hi\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(


In [16]:
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

model = joblib.load('adaboost_decision_tree_model.pkl')
scaler = StandardScaler()

label_encoder = LabelEncoder()

new_user_data = {
    'CreditScore': 0.60,
    'Gender': 'Female',
    'Age': 60,
    'Tenure': 1,
    'Balance': 100,
    'NumOfProducts': 1,
    'HasCrCard': 'No',
    'IsActiveMember': 'N0',
    'EstimatedSalary': 50
}
gender_encoded = label_encoder.fit_transform(['Male', 'Female']) 
gender_value = label_encoder.transform([new_user_data['Gender']])[0]

has_cr_card = 1 if new_user_data['HasCrCard'] == 'Yes' else 0
is_active_member = 1 if new_user_data['IsActiveMember'] == 'Yes' else 0

user_input = pd.DataFrame({
    'CreditScore': [new_user_data['CreditScore']],
    'Gender': [gender_value],
    'Age': [new_user_data['Age']],
    'Tenure': [new_user_data['Tenure']],
    'Balance': [new_user_data['Balance']],
    'NumOfProducts': [new_user_data['NumOfProducts']],
    'HasCrCard': [has_cr_card],
    'IsActiveMember': [is_active_member],
    'EstimatedSalary': [new_user_data['EstimatedSalary']]
})

numerical_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
user_input[numerical_columns] = scaler.fit_transform(user_input[numerical_columns])

prediction = model.predict(user_input)

if prediction[0] == 1:
    print("Customer will exit")
else:
    print("Customer will not exit")


Customer will exit


C:\Users\Hi\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(
